In [9]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy
from bokeh.plotting import figure, output_file, show, save
from bokeh.io import output_notebook, export_png, export_svgs

In [10]:
calc = tc.Calculator(records = tc.Records(), policy = tc.Policy())
calc.advance_to_year(2013)
calc.calc_all()
np.where((calc.records.c17000 > 0) & (calc.records.c04470 > 0), calc.records.s006, 0).sum() 

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


7607134.8599999994

In [514]:
# Maybe change groupfig to handle different functions-- create w_sum to get total number of people, etc. This might not be necessary
# maybe include a last appended stat to the end of each groupfig output-- average for the entire population
# Figure out how to handle negative agis

In [515]:
year = 2015
law = 'pre_tcja_10perc'
output_notebook()
exceldf = pd.DataFrame()
path = '..\\dropbox\\MedicalDeductions\\med_ded_output\\' + str(year) + '_'
writer = pd.ExcelWriter(path + law + '.xlsx')
print(law)

Loading BokehJS ...

pre_tcja_10perc


In [516]:
# Reform
path_name = 'C:\\Users\\derrick.choe\\Tax-Calculator\\taxcalc\\reforms\\2017_law.json'
jfile = open(path_name, 'r')
jtext = jfile.read()
jfile.close()
if law == 'pre_tcja_7hperc':
    reform = tc.Calculator._read_json_policy_reform_text(jtext, {}, {})
    reform[year]['_ID_Medical_frt'] = [0.075]
if law == 'tcja':
    reform = {year: {}}
if law == 'tcja_no_st':
    reform = {year: {'_STD': [[6350.00, 12700.00, 6350.00, 9350.00, 12700.00]]}}
if law == 'pre_tcja_10perc':
    reform = tc.Calculator._read_json_policy_reform_text(jtext, {}, {})
#     reform[year]['_ID_Medical_frt'] = [0.1]



pol = tc.Policy()
pol.implement_reform(reform)

In [517]:
def taxdf(calc, namelist, varlist):
    calcu=copy.deepcopy(calc)
    tdf=pd.DataFrame()
    for v in range (0, len(namelist)):
        tdf[namelist[v]]=getattr(calcu.records,varlist[v])
    return tdf

In [518]:
def createcalc(year, records=tc.Records(), policy=tc.Policy()):
        calc=tc.Calculator(records=records, policy=policy)
        calc.advance_to_year(year)
        calc.calc_all()
        return calc

In [519]:
def w_avg(df, var, wtvar = 's006'):
    return (df[var] * df[wtvar]).sum() / df[wtvar].sum()

In [520]:
def bin_means(df, var, wtvar = 's006'):
    graphlist = []
    declist = []
    for b in range(1, 11):
        subpop = copy.deepcopy(df[df['bins'] == b])
        graphlist.append( w_avg(subpop, var, wtvar))
        declist.append("Decile "+ str(b))
    return [graphlist, declist]

In [521]:
def groupfig(df, var, title = "", wtvar = 's006'):
    graphlist = bin_means(df, var, wtvar)[0]
    declist = bin_means(df, var, wtvar)[1]
    f = figure(title = title)
    f.xaxis.axis_label = 'AGI Bins: ' + str(1) + " through " + str(10)
    f.line(range(1, 11), graphlist)
#     show(f)
    exceldf[title] = graphlist
    return list(zip(declist, graphlist))

In [522]:
def groupfig_med(df, var, title = "", wtvar = 's006'):
    graphlist = []
    declist = []
    for b in range(1, 11):
        subpop = copy.deepcopy(df[df['bins'] == b].sort_values(var))
        subpop['csum'] = np.cumsum(subpop[wtvar])
        graphlist.append(subpop[subpop['csum'] < subpop['csum'].max() / 2].iloc[-1][var])
        declist.append("Decile "+ str(b))
    f = figure(title = title)
    f.xaxis.axis_label = 'AGI Bins: ' + str(1) + " through " + str(10)
    f.line(range(1,11), graphlist)
#     show(f)
    exceldf[title] = graphlist
    return list(zip(declist, graphlist))

In [523]:
vlist=['c00100', 'standard', 'c04470', 'iitax','c19700', 'c17000','e17500', 'e19800','e19200', 'c19200','e20100','combined','MARS','c18300','mtr_inctax','s006']

In [524]:
# Baseline calculator for bins
calc = createcalc(year = year, records = tc.Records(), policy = tc.Policy())
calcdf = taxdf(calc, vlist, vlist)
calcdf_pos = copy.deepcopy(calcdf[calcdf['c00100'] > 5000])
tc.add_quantile_bins(calcdf_pos, income_measure = 'c00100', num_bins = 10)
print("loaded")

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
loaded


In [525]:
#Baseline Calculator
calc_base=createcalc(year = year, records = tc.Records(), policy = pol)

tdf_og = taxdf(calc_base, vlist, vlist)
# Calculate MTR for original calc
tdf_og['mtr'] = calc_base.mtr()[1]

mydf = copy.deepcopy(tdf_og)
mydf['bins'] = calcdf_pos['bins'].map(int)

# Create weight variable for population taking medical deduction
mydf['mdwt'] = np.where((mydf['c17000'] > 0) & (mydf['c04470'] > 0) , mydf['s006'], 0)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [526]:
# Reform without Medical Expense Deduction
reform_m = copy.deepcopy(reform)
# reform_m[year]["_ID_Medical_hc"] = [1]
pol_m = tc.Policy()
pol_m.implement_reform(reform_m)

# Calc without Medical Expense Deduction
calc_nomed = createcalc(year = year, records = tc.Records(), policy = pol_m)

# DF without Medical Expense Deduction
mydf_ref = taxdf(calc_nomed, vlist, vlist)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [527]:
# # 2017 Reform with 7.5% rate
# reform2017s = copy.deepcopy(reform)
# reform2017s[year]['_ID_Medical_frt'] = [0.075]
# pol2017s = tc.Policy()
# pol2017s.implement_reform(reform2017s)

# # Calc
# calc_2017s = createcalc(year = year, records = tc.Records(), policy = pol2017s)

# # DF
# tempdf = taxdf(calc_2017s, vlist, vlist)
# mydf_ref_s = copy.deepcopy(tempdf[tempdf['c00100'] > 0])

# # ADD BINS
# mydf_ref_s['bins'] = mydf['bins']
# mydf_ref_s['mdwt'] = np.where((mydf_ref_s['c17000'] > 0) & (mydf_ref_s['c04470'] > 0) , mydf_ref_s['s006'], 0)

In [528]:
# Create difference in income tax liability var
mydf['diff'] = mydf_ref['iitax'] - mydf['iitax']
mydf['w_diff'] = mydf['diff'] * mydf['s006']
mydf['w_diffck'] = np.where(mydf['mdwt'] > 0, mydf['w_diff'], 0)

In [529]:
basic = {}
subdf = copy.deepcopy(mydf[mydf['c00100'] > 5000])
basic['# Itemizers'] = subdf[subdf['c04470'] > 0]['s006'].sum()
basic['# Taking Med Deduction'] = subdf['mdwt'].sum()
basic['Total Tax Break'] = subdf['w_diff'].sum()
# basic['Total Tax Break Check'] = subdf['w_diffck'].sum()
bdf = pd.DataFrame([basic])
bdf.to_excel(writer, sheet_name= 'General Stats')

# 2: Taxpayer Deciles: Plots

In [530]:
# mydf['deduct_medical'] = np.where((mydf['c17000'] > 0) & (mydf['c04470'] > 0), 1, 0)
# mydf_ref_s['deduct_medical'] = np.where((mydf_ref_s['c17000'] > 0) & (mydf_ref_s['c04470'] > 0), 1, 0)

# l10 = bin_means(mydf, 'deduct_medical')[0]
# l7h = bin_means(mydf_ref_s, 'deduct_medical')[0]

# f = figure(x_range = list(map(str,range(1,11))), title = 'Fraction Who Deduct Medical Expenses', x_axis_label = 'AGI Decile')
# f.vbar(x = np.linspace(0.35, 9.35, 10), top = l10, width = 0.25, legend = 'Rate = 10%')
# f.vbar(x = np.linspace(0.65, 9.65, 10), top = l7h, width = 0.25, color = 'grey', legend = 'Rate = 7.5%')
# f.legend.location = 'top_left'

# show(f)

In [531]:
rlist = list(np.linspace(-.005, 0.995, 51)) + [9e99]
glist = []
for i in range(0, len(rlist) - 1):
    mydf['temp'] = np.where(((rlist[i] * mydf['c00100']) < mydf['e17500']) & (mydf['e17500'] <= (rlist[i + 1] * mydf['c00100'])) & 
                            (mydf['c04470'] > 0), 1, 0)
    glist.append(w_avg(mydf, 'temp','mdwt'))
f = figure(title = 'Fraction of Deduction Takers with Med Expenses Between X Percent AGI: 2% Bins', 
          x_axis_label = 'Fraction of AGI')
f.vbar([x + (rlist[-2] - rlist[0] - 1) / ((len(rlist) - 1) * 2) for x in rlist[:-1]], width = 0.01, top = glist)
show(f)
# export_png(f, filename = 'med_ded_output\\Fraction of Tax Filers with Med Expenses Between X Percent AGI: 1% Bins.png')
histdf = pd.DataFrame()
histdf['Fraction of Deduction Takers with Med Expenses Between X Percent AGI: 1% Bins'] = glist
histdf['Percent Floor'] = rlist[:-1]
histdf.to_excel(writer, sheet_name='Hist Stats')


In [532]:
mydf['deduct_medical'] = np.where((mydf['c17000'] > 0) & (mydf['c04470'] > 0), 1, 0)
groupfig(mydf, 'deduct_medical', 'Fraction Who Take Medical Deduction')

[('Decile 1', 0.00080885568582033567),
 ('Decile 2', 0.021373306797246892),
 ('Decile 3', 0.035887298645846825),
 ('Decile 4', 0.051004684931648489),
 ('Decile 5', 0.063391179656916666),
 ('Decile 6', 0.086452742185254394),
 ('Decile 7', 0.095119014214202657),
 ('Decile 8', 0.090536216251749374),
 ('Decile 9', 0.063705873039101094),
 ('Decile 10', 0.027305822908338384)]

In [533]:
mydf['frac_med'] = mydf['e17500'] / mydf['c00100']
groupfig(mydf, 'frac_med', 'Medical Expenses as Fraction of AGI: Population Taking Medical Deduction', 'mdwt')

[('Decile 1', 0.5406049914754657),
 ('Decile 2', 0.58144644280928992),
 ('Decile 3', 0.61854846282932729),
 ('Decile 4', 0.42012541887516081),
 ('Decile 5', 0.36922223805320492),
 ('Decile 6', 0.27188104374306965),
 ('Decile 7', 0.2301732801323054),
 ('Decile 8', 0.19295440081983345),
 ('Decile 9', 0.16629698699884907),
 ('Decile 10', 0.17017086582074534)]

In [534]:
# Create groupby dataframe for sum of total tax breaks
mygb = pd.DataFrame(mydf.groupby('bins').sum())['w_diff'].reset_index()
mygb['s006'] = [1] * len(mygb)
groupfig(mygb, 'w_diff', 'Total Tax Breaks through Medical Expense Deduction: By Decile')

[('Decile 1', 0.0),
 ('Decile 2', 0.0),
 ('Decile 3', 0.0),
 ('Decile 4', 0.0),
 ('Decile 5', 0.0),
 ('Decile 6', 0.0),
 ('Decile 7', 0.0),
 ('Decile 8', 0.0),
 ('Decile 9', 0.0),
 ('Decile 10', 0.0)]

In [535]:
groupfig(mydf, 'diff', 'Average Tax Break through Medical Expense Deduction (for those taking)', 'mdwt')

[('Decile 1', 0.0),
 ('Decile 2', 0.0),
 ('Decile 3', 0.0),
 ('Decile 4', 0.0),
 ('Decile 5', 0.0),
 ('Decile 6', 0.0),
 ('Decile 7', 0.0),
 ('Decile 8', 0.0),
 ('Decile 9', 0.0),
 ('Decile 10', 0.0)]

In [536]:
groupfig_med(mydf, 'diff', 'Median Tax Break through Medical Expense Deduction (for those taking)', 'mdwt')

[('Decile 1', 0.0),
 ('Decile 2', 0.0),
 ('Decile 3', 0.0),
 ('Decile 4', 0.0),
 ('Decile 5', 0.0),
 ('Decile 6', 0.0),
 ('Decile 7', 0.0),
 ('Decile 8', 0.0),
 ('Decile 9', 0.0),
 ('Decile 10', 0.0)]

# 3: Medical Expense Deduction and Interest Deduction

In [537]:
mydf['deduct_interest'] = np.where((mydf['c19200'] > 0) & (mydf['c04470'] > 0), 1, 0)
groupfig(mydf, 'deduct_interest', 'Fraction Who Take Interest Deduction')

[('Decile 1', 0.00035096384479300018),
 ('Decile 2', 0.017066349753192052),
 ('Decile 3', 0.033441839669365266),
 ('Decile 4', 0.066186494231649101),
 ('Decile 5', 0.12013966568682125),
 ('Decile 6', 0.20847921694069468),
 ('Decile 7', 0.30924093587810036),
 ('Decile 8', 0.43627111585806189),
 ('Decile 9', 0.62084732533630482),
 ('Decile 10', 0.78508218382357631)]

In [538]:
groupfig(mydf, 'deduct_interest', 'Fraction Who Take Interest Deduction: Population Taking Medical Deduction', 'mdwt')

[('Decile 1', 0.14350519270713133),
 ('Decile 2', 0.42146833859088678),
 ('Decile 3', 0.40276058371976409),
 ('Decile 4', 0.48355660287053948),
 ('Decile 5', 0.52084081659802517),
 ('Decile 6', 0.56512411768357418),
 ('Decile 7', 0.58319048358636927),
 ('Decile 8', 0.66550542952823755),
 ('Decile 9', 0.65730480753180098),
 ('Decile 10', 0.73288666630175625)]

# 4: Tax Savings Analysis

In [539]:
mydf['diff_frac'] = mydf['diff'] / mydf['e17500']
groupfig(mydf, 'diff_frac', 'Federal Cost Sharing (Tax Break div. Total Medical Expenses): Population taking Med. Ded', 'mdwt')

[('Decile 1', 0.0),
 ('Decile 2', 0.0),
 ('Decile 3', 0.0),
 ('Decile 4', 0.0),
 ('Decile 5', 0.0),
 ('Decile 6', 0.0),
 ('Decile 7', 0.0),
 ('Decile 8', 0.0),
 ('Decile 9', 0.0),
 ('Decile 10', 0.0)]

In [540]:
groupfig_med(mydf, 'diff_frac', 'Median Federal Cost Sharing: Population taking Med. Ded.', 'mdwt')

[('Decile 1', 0.0),
 ('Decile 2', 0.0),
 ('Decile 3', 0.0),
 ('Decile 4', 0.0),
 ('Decile 5', 0.0),
 ('Decile 6', 0.0),
 ('Decile 7', 0.0),
 ('Decile 8', 0.0),
 ('Decile 9', 0.0),
 ('Decile 10', 0.0)]

In [541]:
mydf['mtrx100'] = 100 * mydf['mtr']
groupfig(mydf, 'mtrx100', 'MTR (Avg Tax savings from add. $100 Income): Population Taking Medical Deduction', 'mdwt')

[('Decile 1', 7.7219769988016349),
 ('Decile 2', 3.5166882177707),
 ('Decile 3', 7.8579071740013076),
 ('Decile 4', 12.261123768371375),
 ('Decile 5', 15.475505620801613),
 ('Decile 6', 17.613499792673991),
 ('Decile 7', 19.237230579553),
 ('Decile 8', 18.005129068250366),
 ('Decile 9', 22.483623337708877),
 ('Decile 10', 27.918515485862493)]

In [542]:
# Check 10th decile
# why is dollar value so high given relatively low number of people
# get medians as well
# what other types of interest paid
# Do for 7.5% cutoff
# email Brill

In [543]:
blist = []
poplist = []
medlist = []
for b in range(1, 11):
    blist.append(mydf[mydf['bins'] == b]['c00100'].min())
    poplist.append(mydf[mydf['bins'] == b]['s006'].sum())
    medlist.append(mydf[mydf['bins'] == b]['mdwt'].sum())

exceldf['Bin AGI Floor'] = blist
exceldf['Bin Population'] = poplist
exceldf['Number Taking Med Deduction'] = medlist
exceldf

,Fraction Who Take Medical Deduction,Medical Expenses as Fraction of AGI: Population Taking Medical Deduction,Total Tax Breaks through Medical Expense Deduction: By Decile,Average Tax Break through Medical Expense Deduction (for those taking),Median Tax Break through Medical Expense Deduction (for those taking),Fraction Who Take Interest Deduction,Fraction Who Take Interest Deduction: Population Taking Medical Deduction,Federal Cost Sharing (Tax Break div. Total Medical Expenses): Population taking Med. Ded,Median Federal Cost Sharing: Population taking Med. Ded.,MTR (Avg Tax savings from add. $100 Income): Population Taking Medical Deduction,Bin AGI Floor,Bin Population,Number Taking Med Deduction
0,0.000809,0.540605,0.0,0.0,0.0,0.000351,0.143505,0.0,0.0,7.721977,5000.198373,13392376.65,10832.50
1,0.021373,0.581446,0.0,0.0,0.0,0.017066,0.421468,0.0,0.0,3.516688,10240.174162,13394730.76,286289.69
2,0.035887,0.618548,0.0,0.0,0.0,0.033442,0.402761,0.0,0.0,7.857907,15796.111989,13395088.74,480713.55
3,0.051005,0.420125,0.0,0.0,0.0,0.066186,0.483557,0.0,0.0,12.261124,22623.596399,13393245.56,683118.27
4,0.063391,0.369222,0.0,0.0,0.0,0.120140,0.520841,0.0,0.0,15.475506,30558.591897,13394874.88,849116.92
5,0.086453,0.271881,0.0,0.0,0.0,0.208479,0.565124,0.0,0.0,17.613500,41106.921628,13393848.37,1157934.92
6,0.095119,0.230173,0.0,0.0,0.0,0.309241,0.583190,0.0,0.0,19.237231,54645.473251,13393765.70,1274001.79
7,0.090536,0.192954,0.0,0.0,0.0,0.436271,0.665505,0.0,0.0,18.005129,72018.714433,13394713.19,1212706.65
8,0.063706,0.166297,0.0,0.0,0.0,0.620847,0.657305,0.0,0.0,22.483623,99668.430960,13393540.49,853247.19
9,0.027306,0.170171,0.0,0.0,0.0,0.785082,0.732887,0.0,0.0,27.918515,148143.519963,13394649.97,365751.94


In [544]:
exceldf.to_excel(writer, sheet_name='Decile Stats')
writer.save()